In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\feature_selection.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Preprosessed\Preprosessedd.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,0,26,4388,2187,0.988754,34727,686,16,4190000,1909505035,...,2,2025,10,20,0,0.166667,0.008288,0.020214,0.000164,0.000004
1,1,1447,2374,4507,0.898341,45151,1514,21,188000,135785655,...,2,2025,10,16,3,0.666667,0.240164,0.033996,0.008053,0.000112
2,2,3989,2973,4820,0.910707,353713,6485,127,37300,54921887,...,2,2025,10,17,4,0.000000,9.482668,0.018693,0.173856,0.003405
3,3,7437,7301,3101,0.944730,2171652,78431,183,43800,88085601,...,1,2025,10,18,5,0.333333,49.579964,0.036200,1.790621,0.004178
4,4,2123,7961,1298,0.892396,246429,9321,949,2490000,501089468,...,0,2025,10,16,3,0.666667,0.098967,0.041675,0.003743,0.000381


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          8493 non-null   int64  
 1   Video ID            8493 non-null   int64  
 2   Video Title         8493 non-null   int64  
 3   Channel             8493 non-null   int64  
 4   Published Date      8493 non-null   float64
 5   Views               8493 non-null   int64  
 6   Likes               8493 non-null   int64  
 7   Comments            8493 non-null   int64  
 8   Subscribers         8493 non-null   int64  
 9   Channel Views       8493 non-null   int64  
 10  Country             8493 non-null   int64  
 11  Region              8493 non-null   int64  
 12  CategoryID          8493 non-null   int64  
 13  CategoryName        8493 non-null   int64  
 14  Cluster             8493 non-null   int64  
 15  CategoryID_enc      8493 non-null   int64  
 16  Countr

In [4]:
import pandas as pd
import plotly.express as px

# Raqamli ustunlarni tanlash
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# faqat raqamli ustunlar bo'yicha korrelyatsiya
corr_matrix = df[numeric_cols].corr()

# CategoryName ustunining boshqa ustunlar bilan korrelyatsiyasi
CategoryName_corr = corr_matrix[['CategoryName']].sort_values(by='CategoryName', ascending=False)

# Heatmap uchun DataFrame ni transformatsiya qilish
heatmap_data = CategoryName_corr.reset_index()
heatmap_data.columns = ['Feature', 'Correlation_with_CategoryName']

# Plotly Express yordamida bar grafik
fig = px.bar(
    heatmap_data,
    x='Feature',
    y='Correlation_with_CategoryName',
    title='CategoryName bilan boshqa raqamli ustunlar o\'rtasidagi korrelyatsiya',
    labels={'Correlation_with_CategoryName': 'Korrelyatsiya qiymati', 'Feature': 'Xususiyatlar'}
)

fig.show()


In [5]:
import numpy as np
import plotly.figure_factory as ff

# Qiymatlarni 2 xonagacha qisqartirish
annot_text = np.around(corr_matrix.values, decimals=2).astype(str)

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=annot_text,
    colorscale='Viridis',
    showscale=True
)

fig.update_layout(title='Raqamli ustunlar o\'rtasidagi korrelyatsiya matritsasi')
fig.show()

# Feature Selection Filtering 

In [6]:
import seaborn as sns               
import matplotlib.pyplot as plt     


corr_matrix = df.corr().abs()  

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  


to_drop = [col for col in upper.columns if any(upper[col] > 0.8)]  
corr_matrix = df.corr().abs() 
print(corr_matrix)  


                    Unnamed: 0  Video ID  Video Title   Channel  \
Unnamed: 0            1.000000  0.003286     0.007227  0.009950   
Video ID              0.003286  1.000000     0.003645  0.001701   
Video Title           0.007227  0.003645     1.000000  0.159717   
Channel               0.009950  0.001701     0.159717  1.000000   
Published Date        0.002673  0.015379     0.001231  0.007830   
Views                 0.005807  0.015091     0.028342  0.000031   
Likes                 0.007851  0.005348     0.028292  0.001984   
Comments              0.010224  0.001029     0.014110  0.010213   
Subscribers           0.002241  0.001356     0.016549  0.002822   
Channel Views         0.002554  0.002265     0.012593  0.011309   
Country               0.024975  0.006360     0.024125  0.010894   
Region                0.001248  0.012039     0.035045  0.014039   
CategoryID            0.004159  0.006255     0.019482  0.007491   
CategoryName          0.006378  0.005755     0.006783  0.02159

In [7]:

import numpy as np   

corr_matrix = df.corr().abs()  

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  

to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]  

df_filtered = df.drop(columns=to_drop)  
print("\nOriginal shape:", df.shape)  

print("Shape after dropping correlated features:", df_filtered.shape) 



Original shape: (8493, 33)
Shape after dropping correlated features: (8493, 30)


In [8]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          8493 non-null   int64  
 1   Video ID            8493 non-null   int64  
 2   Video Title         8493 non-null   int64  
 3   Channel             8493 non-null   int64  
 4   Published Date      8493 non-null   float64
 5   Views               8493 non-null   int64  
 6   Likes               8493 non-null   int64  
 7   Comments            8493 non-null   int64  
 8   Subscribers         8493 non-null   int64  
 9   Channel Views       8493 non-null   int64  
 10  Country             8493 non-null   int64  
 11  Region              8493 non-null   int64  
 12  CategoryID          8493 non-null   int64  
 13  CategoryName        8493 non-null   int64  
 14  Cluster             8493 non-null   int64  
 15  CategoryID_enc      8493 non-null   int64  
 16  VideoC

In [9]:
# Yuqori korrelyatsiyaga ega juftliklarni saqlash uchun bo‘sh ro‘yxat yaratamiz
high_corr_pairs = []


for col in upper.columns:
    
    for row in upper.index:
        
        if upper.loc[row, col] is not np.nan and upper.loc[row, col] > 0.95:
            
            high_corr_pairs.append([row, col, round(upper.loc[row, col], 2)])


corr_table = pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

print("Highly correlated feature pairs (correlation > 0.95):")
print(corr_table)

Highly correlated feature pairs (correlation > 0.95):
            Feature 1           Feature 2  Correlation
0             Country         Country_enc          1.0
1   Country_Cluster_x   Country_Cluster_y          1.0
2  Category_Cluster_x  Category_Cluster_y          1.0


In [10]:
import plotly.express as px  

corr_long = corr_matrix.reset_index().melt(id_vars='index')  

corr_long.columns = ['Feature 1', 'Feature 2', 'Correlation']  

fig = px.imshow(
    corr_matrix,                  
    text_auto='.2f',              
    aspect="auto",                
    color_continuous_scale='RdBu_r',  
    zmin=-1, zmax=1,             
    title="Correlation Matrix Heatmap (Interactive)"  
)


fig.update_layout(
    width=900,          
    height=800,          
    xaxis_title="Features",  
    yaxis_title="Features"   
)

fig.show()

# Low Variance

In [11]:
# Low Variance
from sklearn.feature_selection import VarianceThreshold  
threshold = 0.01  
selector = VarianceThreshold(threshold=threshold)  
selector.fit(df_filtered)  
low_variance_features = df_filtered.columns[~selector.get_support()] 
df_low_variance_filtered = df_filtered.drop(columns=low_variance_features)  
print("\nOriginal shape:", df_filtered.shape)  
print("Shape after dropping low-variance features:", df_low_variance_filtered.shape)  


Original shape: (8493, 30)
Shape after dropping low-variance features: (8493, 28)


In [12]:
# Past variansli ustunlarni aniqlash:

low_variance_features = df_filtered.columns[~selector.get_support()]
print("Features with low variance (to drop):", list(low_variance_features))

Features with low variance (to drop): ['Year', 'Engagement']


In [13]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
variances = df[numeric_cols].var()
threshold = 0.01
low_variance_features = variances[variances < threshold].index.tolist()
var_df = pd.DataFrame({
    'Feature': variances.index,
    'Variance': variances.values,
    'LowVariance': ['Yes' if f in low_variance_features else 'No' for f in variances.index]
})


fig = px.bar(
    var_df,
    x='Feature',        
    y='Variance',       
    color='LowVariance',  
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Variance',    
    title='Feature Variance (Low-Variance Highlighted in Red)'  
)


fig.update_layout(
    xaxis_tickangle=-45,
    width=1000,
    height=600
)

fig.show()

In [14]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          8493 non-null   int64  
 1   Video ID            8493 non-null   int64  
 2   Video Title         8493 non-null   int64  
 3   Channel             8493 non-null   int64  
 4   Published Date      8493 non-null   float64
 5   Views               8493 non-null   int64  
 6   Likes               8493 non-null   int64  
 7   Comments            8493 non-null   int64  
 8   Subscribers         8493 non-null   int64  
 9   Channel Views       8493 non-null   int64  
 10  Country             8493 non-null   int64  
 11  Region              8493 non-null   int64  
 12  CategoryID          8493 non-null   int64  
 13  CategoryName        8493 non-null   int64  
 14  Cluster             8493 non-null   int64  
 15  CategoryID_enc      8493 non-null   int64  
 16  VideoC

In [15]:
df_filtered.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Feature_Selection\Filtered_Featuress.csv")

# Feature Selection Wrapper 

 # RFE + Linear Regression

In [16]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

X = df.drop('CategoryName', axis=1)
y = df['CategoryName']

# RFE + Linear Regression 

In [17]:
# Oddiy chiziqli regressiya modelini yaratamiz (bu model RFE uchun asos bo'ladi)
model = LinearRegression()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)
selected_features = X.columns[rfe.support_]
print("Selected features:", selected_features)

Selected features: Index(['Channel', 'Views', 'Likes', 'Comments', 'Subscribers', 'Channel Views',
       'Country', 'CategoryID', 'Cluster', 'CategoryID_enc', 'Country_enc',
       'VideoCluster', 'Video_Cluster', 'Country_Cluster_x',
       'Category_Cluster_x', 'Country_Cluster_y', 'Category_Cluster_y', 'Day',
       'DayOfWeek', 'Engagement'],
      dtype='object')


# RFE + RandomForestRegressor

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

model = RandomForestRegressor()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]
print("Selected features (RandomForest):", selected_features)


Selected features (RandomForest): Index(['Unnamed: 0', 'Video ID', 'Video Title', 'Channel', 'Published Date',
       'Views', 'Likes', 'Comments', 'Subscribers', 'Channel Views',
       'CategoryID', 'CategoryID_enc', 'Country_enc', 'Category_Cluster_x',
       'Category_Cluster_y', 'Views_per_Sub', 'Engagement', 'Like_per_Sub',
       'Comment_per_Sub', 'Published_ts'],
      dtype='object')


# RFE + Decision Tree Regressor 

In [19]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]
print("Selected features (DecisionTree):", selected_features)


Selected features (DecisionTree): Index(['Unnamed: 0', 'Video ID', 'Channel', 'Published Date', 'Views', 'Likes',
       'Comments', 'Subscribers', 'Channel Views', 'Country', 'CategoryID',
       'CategoryID_enc', 'Country_enc', 'Category_Cluster_y', 'Day',
       'DayOfWeek', 'Views_per_Sub', 'Engagement', 'Comment_per_Sub',
       'Published_ts'],
      dtype='object')


# Feature Selection Embedded + Lasso CV

In [20]:
# Xususiyat tanlash uchun Linear Model'larda ishlatiladigan usul (LASSO regression)

# LassoCV (L1-regularizatsiya) ni import qilamiz
# CV - cross-validation, u model uchun optimal alpha (regulyarizatsiya kuchi) tanlab beradi
from sklearn.linear_model import LassoCV
X = df.drop('CategoryName', axis=1)
y = df['CategoryName']
lasso = LassoCV(cv=10, random_state=42).fit(X, y)
importance = np.abs(lasso.coef_)
selected_features = X.columns[importance > 0]
print("Selected features using Lasso (non-zero coefficients):")
print(selected_features.tolist())

Selected features using Lasso (non-zero coefficients):
['Published_ts']


In [21]:
# Faqat muhim featurelarni saqlab qolish
percentile_threshold = np.percentile(importance, 90) 
top_features = X.columns[importance >= percentile_threshold]
print("\nTop 25% important features based on Lasso coefficients:")
print(top_features.tolist())


Top 25% important features based on Lasso coefficients:
['Unnamed: 0', 'Video ID', 'Video Title', 'Channel', 'Published Date', 'Views', 'Likes', 'Comments', 'Subscribers', 'Channel Views', 'Country', 'Region', 'CategoryID', 'Cluster', 'CategoryID_enc', 'Country_enc', 'VideoCluster', 'Channel_Cluster', 'Video_Cluster', 'Country_Cluster_x', 'Category_Cluster_x', 'Country_Cluster_y', 'Category_Cluster_y', 'Year', 'Month', 'Day', 'DayOfWeek', 'DayName', 'Views_per_Sub', 'Engagement', 'Like_per_Sub', 'Comment_per_Sub', 'Published_ts']


In [22]:
# Feature importance'ni jadval shaklida tayyorlash

feat_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
})

# Importance qiymatlariga qarab kamayish tartibida (eng muhimdan kam muhimga qarab) saralaymiz
feat_df = feat_df.sort_values(by='Importance', ascending=False)
percentile_threshold = np.percentile(importance, 85)
feat_df['Top15%'] = np.where(feat_df['Importance'] >= percentile_threshold, 'Yes', 'No')
print("Feature Importance Table:")
print(feat_df)

Feature Importance Table:
               Feature    Importance Top15%
32        Published_ts  5.494183e-07    Yes
0           Unnamed: 0  0.000000e+00    Yes
1             Video ID  0.000000e+00    Yes
3              Channel  0.000000e+00    Yes
2          Video Title  0.000000e+00    Yes
5                Views  0.000000e+00    Yes
6                Likes  0.000000e+00    Yes
7             Comments  0.000000e+00    Yes
4       Published Date  0.000000e+00    Yes
9        Channel Views  0.000000e+00    Yes
10             Country  0.000000e+00    Yes
11              Region  0.000000e+00    Yes
12          CategoryID  0.000000e+00    Yes
13             Cluster  0.000000e+00    Yes
14      CategoryID_enc  0.000000e+00    Yes
15         Country_enc  0.000000e+00    Yes
8          Subscribers  0.000000e+00    Yes
16        VideoCluster  0.000000e+00    Yes
17     Channel_Cluster  0.000000e+00    Yes
19   Country_Cluster_x  0.000000e+00    Yes
18       Video_Cluster  0.000000e+00    Yes
21   C

In [23]:
fig = px.bar(
    feat_df,                
    x='Feature',            
    y='Importance',         
    color='Top15%',         
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Importance',      
    title='Lasso Feature Importance'  
)


fig.update_layout(
    xaxis_tickangle=-45,  
    width=1000,           
    height=600            
)

fig.show()

# Feature Selection Embedded + Random Forest Regression 

In [24]:
# Random Forest Regressorni import qilamiz
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500, random_state=42)

rf.fit(X, y)

importances = rf.feature_importances_
feat_df = pd.DataFrame({
    'Feature': X.columns,       
    'Importance': importances   
})
feat_df = feat_df.sort_values(by='Importance', ascending=False)
percentile_threshold = np.percentile(importances, 85)
feat_df['Top15%'] = np.where(feat_df['Importance'] >= percentile_threshold, 'Yes', 'No')
print("Random Forest Feature Importance Table:")
print(feat_df)

Random Forest Feature Importance Table:
               Feature    Importance Top15%
12          CategoryID  4.992266e-01    Yes
14      CategoryID_enc  3.679482e-01    Yes
22  Category_Cluster_y  6.782183e-02    Yes
20  Category_Cluster_x  6.500340e-02    Yes
30        Like_per_Sub  9.717529e-16    Yes
8          Subscribers  9.694972e-16     No
2          Video Title  9.602479e-16     No
32        Published_ts  9.596209e-16     No
9        Channel Views  9.393516e-16     No
1             Video ID  9.284413e-16     No
28       Views_per_Sub  9.168453e-16     No
0           Unnamed: 0  9.145345e-16     No
4       Published Date  9.104732e-16     No
3              Channel  8.703656e-16     No
29          Engagement  8.441524e-16     No
5                Views  8.273961e-16     No
6                Likes  8.008994e-16     No
31     Comment_per_Sub  7.174852e-16     No
7             Comments  6.161732e-16     No
15         Country_enc  4.810968e-16     No
10             Country  3.569709e-16

In [25]:
# Plotly Express yordamida Random Forest feature importance'ni grafik ko'rinishida chiqaramiz
fig = px.bar(
    feat_df,                
    x='Feature',            
    y='Importance',         
    color='Top15%',         
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Importance',      
    title='Random Forest Feature Importance'  
)


fig.update_layout(
    xaxis_tickangle=-45,  
    width=1000,           
    height=600            
)

fig.show()

# Feature Selection Embedded + LassoCV  

In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# (1) Ma'lumotlarni standartlashtirish juda muhim (Lasso uchun shart)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# (2) LassoCV modelini yaratamiz
lasso = LassoCV(cv=5, random_state=42)

# (3) Modelni o‘qitamiz
lasso.fit(X_scaled, y)

# (4) Feature importance (koeffitsientlarning mutlaq qiymati)
importances = np.abs(lasso.coef_)

# (5) DataFrame ko‘rinishida natijalar
feat_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# (6) Yuqori 15% (85-percentile dan yuqori) xususiyatlarni belgilang
percentile_threshold = np.percentile(importances, 85)
feat_df['Top15%'] = np.where(feat_df['Importance'] >= percentile_threshold, 'Yes', 'No')

# (7) Natijani chiqaramiz
print("LassoCV Feature Importance Table:")
print(feat_df)


LassoCV Feature Importance Table:
               Feature    Importance Top15%
12          CategoryID  4.716190e-01    Yes
29          Engagement  2.286910e-01    Yes
14      CategoryID_enc  1.708794e-01    Yes
16        VideoCluster  1.232840e-01    Yes
26           DayOfWeek  7.071684e-02    Yes
18       Video_Cluster  6.393421e-02     No
25                 Day  6.066246e-02     No
7             Comments  5.411552e-02     No
6                Likes  5.143802e-02     No
10             Country  4.747209e-02     No
20  Category_Cluster_x  4.386135e-02     No
13             Cluster  3.634538e-02     No
19   Country_Cluster_x  1.056543e-02     No
3              Channel  8.545476e-03     No
8          Subscribers  6.574240e-03     No
11              Region  4.308947e-03     No
22  Category_Cluster_y  4.183108e-17     No
0           Unnamed: 0  0.000000e+00     No
9        Channel Views  0.000000e+00     No
15         Country_enc  0.000000e+00     No
5                Views  0.000000e+00     N